# Water Jug problem 

## General Approach

In [2]:
def bjug(x,y):
    if x==0:return (X,y) # filling the big jug to maximum level
    if y==Y:return (x,0) # empty the jug
    if x>y and x+y>Y: return (x-(Y-y),Y)  # checking the overflow condition
    return (0,x+y) 

def sjug(x,y):
    if y==0: return(x,Y) # filling the small jug to maximum level
    if x==X: return(0,y) # empty the jug
    if x<X and x+y>X: return (X,(x+y)-X) # checking the overflow condition
    return (x+y,0)

def waterjug():
    for i in range(2):
        x=y=c=0
        l=['(0,0)'] # a list to keep track of jug states
        while goal not in (x,y):
            x,y=bjug(x,y) if i==0 else sjug(x,y)
            if f'({x},{y})' in l:
                print('Not possible') # stating the non possible condition to avoid infinite loop
                return
            l.append(f'({x},{y})')
            c+=1
        if i==0: print(f'\nwhile using Big jug of capacity {X} as filler, it took {c} operations to achieve the goal state') 
        else: print(f'\nwhile using small jug of capacity {Y} as filler, it took {c} operations to achieve the goal state')
        print('\n  \u2193 \n'.join(l))

if __name__ == '__main__':
    Y,X=sorted([int(x) for x in input('enter two water jugs capacity space seperated: ').split()])
    goal=int(input('enter goal capacity: '))
    waterjug()

enter two water jugs capacity space seperated: 7 4
enter goal capacity: 3

while using Big jug of capacity 7 as filler, it took 2 operations to achieve the goal state
(0,0)
  ↓ 
(7,0)
  ↓ 
(3,4)

while using small jug of capacity 4 as filler, it took 16 operations to achieve the goal state
(0,0)
  ↓ 
(0,4)
  ↓ 
(4,0)
  ↓ 
(4,4)
  ↓ 
(7,1)
  ↓ 
(0,1)
  ↓ 
(1,0)
  ↓ 
(1,4)
  ↓ 
(5,0)
  ↓ 
(5,4)
  ↓ 
(7,2)
  ↓ 
(0,2)
  ↓ 
(2,0)
  ↓ 
(2,4)
  ↓ 
(6,0)
  ↓ 
(6,4)
  ↓ 
(7,3)


## DFS Approach

In [3]:
class State:
    def __init__(self,bjug,sjug,bmax,smax):
        self.bjug = bjug
        self.sjug = sjug
        self.bmax = bmax
        self.smax = smax
        self.par= None
    
    def isvalid(self): # to check the state is valid or not
        if self.bjug>=0 and self.sjug>=0 and self.bjug<=self.bmax and self.sjug<=self.smax:
            return True
        return False
    
    def isgoal(self,goal): # to check the goal is reached or not
        if self.bjug==goal or self.sjug==goal:
            return True
        return False
    
    def __eq__(self,other): #for checking to objects
        return self.bjug==other.bjug and self.sjug==other.sjug
    
    def __hash__(self): #for set hashing
        return hash((self.bjug,self.sjug))
    
    def __str__(self): #for printing the object
        return f'({self.bjug},{self.sjug})'
    
    
def successors(node,x): # to derive the sucessors of a node
    children = []
    '''from Big jug to Small jug'''
    if x==1: 
        if node.bjug==0:
            nstate=State(node.bmax,node.sjug,node.bmax,node.smax)
            if nstate.isvalid():
                nstate.par=node
                children.append(nstate) 
        if node.sjug==node.smax:
            nstate=State(node.bjug,0,node.bmax,node.smax)
            if nstate.isvalid():
                nstate.par=node
                children.append(nstate)
        if node.bjug>node.sjug and node.bjug+node.sjug>node.smax: 
            nstate=State(node.bjug-(node.smax-node.sjug),node.smax,node.bmax,node.smax)
            if nstate.isvalid():
                nstate.par=node
                children.append(nstate) 
        nstate=State(0,node.bjug+node.sjug,node.bmax,node.smax)
        if nstate.isvalid():
            nstate.par=node
            children.append(nstate)
    else:
        '''from Small jug to Big jug'''
        if node.sjug==0: 
            nstate=State(node.bjug,node.smax,node.bmax,node.smax)
            if nstate.isvalid():
                nstate.par=node
                children.append(nstate) 
        if node.bjug==node.bmax: 
            nstate=State(0,node.sjug,node.bmax,node.smax)
            if nstate.isvalid():
                nstate.par=node
                children.append(nstate) 
        if node.bjug<node.bmax and node.bjug+node.sjug>node.bmax: 
            nstate=State (node.bmax,(node.bjug+node.sjug)-node.bmax,node.bmax,node.smax)
            if nstate.isvalid():
                nstate.par=node
                children.append(nstate) 
        nstate=State (node.bjug+node.sjug,0,node.bmax,node.smax)
        if nstate.isvalid():
            nstate.par=node
            children.append(nstate)
    return children


def dfs(bj,sj,goal,x):
    recordx = State(0,0,bj,sj)
    if recordx.isgoal(goal): return recordx
    openlist = [recordx] #openlist is a list of states
    closed = set() #closed is a list of visited states
    while openlist:
        state = openlist.pop(0)
        if state.isgoal(goal): return state
        closed.add(state)
        children = successors(state,x)
        temp=[]
        for i in children:
            if i not in closed: temp.append(i) 
        openlist = temp[::-1] + openlist #adding children to openlist
    return None

def display(solution,x):
    if solution:
        path = [solution]
        par = solution.par #for parent tracking
        while par:
            path.append(par)
            par = par.par
        for i in path[::-1]: print(i) #printing in reverse order from initial state to final
        if x==1: print(f'While using Big jug of capacity {path[-2].bjug} as filler it took {len(path)-1} steps to reach goal\n')
        else: print(f'While using Small jug of capacity {path[-2].sjug} as filler it took {len(path)-1} steps to reach goal\n')
    else: print('No solution found')

if __name__ == "__main__":
    Y,X=sorted([int(x) for x in input('enter two water jugs capacity space seperated: ').split()])
    goal=int(input('enter goal capacity: '))
    display(dfs(X,Y,goal,1),1)
    display(dfs(X,Y,goal,2),2)

enter two water jugs capacity space seperated: 11 3
enter goal capacity: 8
(0,0)
(11,0)
(8,3)
While using Big jug of capacity 11 as filler it took 2 steps to reach goal

(0,0)
(0,3)
(3,0)
(3,3)
(6,0)
(6,3)
(9,0)
(9,3)
(11,1)
(0,1)
(1,0)
(1,3)
(4,0)
(4,3)
(7,0)
(7,3)
(10,0)
(10,3)
(11,2)
(0,2)
(2,0)
(2,3)
(5,0)
(5,3)
(8,0)
While using Small jug of capacity 3 as filler it took 24 steps to reach goal



## BFS Approach

In [4]:
class State:
    def __init__(self,bjug,sjug,bmax,smax):
        self.bjug = bjug
        self.sjug = sjug
        self.bmax = bmax
        self.smax = smax
        self.par= None
    
    def isvalid(self):
        if self.bjug>=0 and self.sjug>=0 and self.bjug<=self.bmax and self.sjug<=self.smax:
            return True
        return False
    
    def isgoal(self,goal):
        if self.bjug==goal or self.sjug==goal:
            return True
        return False
    
    def __eq__(self,other):
        return self.bjug==other.bjug and self.sjug==other.sjug
    
    def __hash__(self):
        return hash((self.bjug,self.sjug))
    
    def __str__(self):
        return f'({self.bjug},{self.sjug})'
    
    
def successors(node,x):
    children = []
    if x==1:
        if node.bjug==0:
            nstate=State(node.bmax,node.sjug,node.bmax,node.smax)
            if nstate.isvalid():
                nstate.par=node
                children.append(nstate) 
        if node.sjug==node.smax:
            nstate=State(node.bjug,0,node.bmax,node.smax)
            if nstate.isvalid():
                nstate.par=node
                children.append(nstate)
        if node.bjug>node.sjug and node.bjug+node.sjug>node.smax: 
            nstate=State(node.bjug-(node.smax-node.sjug),node.smax,node.bmax,node.smax)
            if nstate.isvalid():
                nstate.par=node
                children.append(nstate) 
        nstate=State(0,node.bjug+node.sjug,node.bmax,node.smax)
        if nstate.isvalid():
            nstate.par=node
            children.append(nstate)
    else:
        if node.sjug==0: 
            nstate=State(node.bjug,node.smax,node.bmax,node.smax)
            if nstate.isvalid():
                nstate.par=node
                children.append(nstate) 
        if node.bjug==node.bmax: 
            nstate=State(0,node.sjug,node.bmax,node.smax)
            if nstate.isvalid():
                nstate.par=node
                children.append(nstate) 
        if node.bjug<node.bmax and node.bjug+node.sjug>node.bmax: 
            nstate=State (node.bmax,(node.bjug+node.sjug)-node.bmax,node.bmax,node.smax)
            if nstate.isvalid():
                nstate.par=node
                children.append(nstate) 
        nstate=State (node.bjug+node.sjug,0,node.bmax,node.smax)
        if nstate.isvalid():
            nstate.par=node
            children.append(nstate)
    return children


def bfs(bj,sj,goal,x):
    recordx = State(0,0,bj,sj)
    if recordx.isgoal(goal): return recordx
    openlist = [recordx] #openlist is a list of states
    closed = set() #closed is a list of visited states
    while openlist:
        state = openlist.pop(0)
        if state.isgoal(goal): return state
        closed.add(state)
        children = successors(state,x)
        for i in children:
            if i not in closed: 
                openlist.append(i)  #adding children to openlist
    return None

def display(solution,x):
    if solution:
        path = [solution]
        par = solution.par #for parent tracking
        while par:
            path.append(par)
            par = par.par
        for i in path[::-1]: print(i) #printing in reverse order from initial state to final
        if x==1: print(f'While using Big jug of capacity {path[-2].bjug} as filler it took {len(path)-1} steps to reach goal\n')
        else: print(f'While using Small jug of capacity {path[-2].sjug} as filler it took {len(path)-1} steps to reach goal\n')
    else: print('No solution found')

if __name__ == "__main__":
    Y,X=sorted([int(x) for x in input('enter two water jugs capacity space seperated: ').split()])
    goal=int(input('enter goal capacity: '))
    display(bfs(X,Y,goal,1),1)
    display(bfs(X,Y,goal,2),2)

enter two water jugs capacity space seperated: 12 5
enter goal capacity: 4
(0,0)
(12,0)
(7,5)
(7,0)
(2,5)
(2,0)
(0,2)
(12,2)
(9,5)
(9,0)
(4,5)
While using Big jug of capacity 12 as filler it took 10 steps to reach goal

(0,0)
(0,5)
(5,0)
(5,5)
(10,0)
(10,5)
(12,3)
(0,3)
(3,0)
(3,5)
(8,0)
(8,5)
(12,1)
(0,1)
(1,0)
(1,5)
(6,0)
(6,5)
(11,0)
(11,5)
(12,4)
While using Small jug of capacity 5 as filler it took 20 steps to reach goal

